In [1]:
import tkinter
from tkinter import Label
import time
from tkinter import filedialog
import FRAUDARESUMEDETECTION as frd
import tkinter
import pandas as pd
import numpy
import string
import os
import pdfplumber
import re
import unicodedata
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from win32com import client
import spacy
import sys,fitz
import en_core_web_sm
import en_core_web_lg
import en_core_web_trf
from gensim.parsing.preprocessing import remove_stopwords
import shutil
from collections import Counter
from docx import Document
from docx.shared import Inches

c:\Users\prashant mishra\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
gui=tkinter.Tk()
gui['bg']='aqua'
gui.geometry('600x400')
directory=None
l1=Label(gui,text="Select the folder :",font=("Times New Roman Bold",15),fg='blue',bg='aqua')
l1.place(x=0,y=0)
def run(directory):
    TotalFiles=0        
    if frd.n_files(directory) == 0:
        print('There are no files to convert')
        exit()
        
    #creating new directory for all pdfs
    if not os.path.exists(directory + '\\PDFs'):
        os.makedirs(directory + '\\PDFs')
    if not os.path.exists(directory + '\\FraudPDFs'):
        os.makedirs(directory + '\\FraudPDFs')

    #     print('Starting conversion... \n')
    try:
        word = client.DispatchEx('Word.Application')
        i=1
        for file in os.listdir(directory):
                #Getting extension of the file
            if (file.endswith('.doc') or file.endswith('.docx')):
                ending = ""
                if file.endswith('.doc'):
                    ending = '.doc'
                if file.endswith('.docx'):
                    ending = '.docx'
                new_name = file.replace(ending,r".pdf")
                in_file = os.path.abspath(directory + '\\' + file)
                new_file = os.path.abspath(directory + '\\PDFs' + '\\' + new_name)
                doc = word.Documents.Open(in_file)
                print(new_name)
                TotalFiles+=1
                doc.SaveAs(new_file,FileFormat = 17)
                doc.Close()
            elif (file.endswith('.pdf')):
                print(file)
                shutil.copy(directory+"\\"+file, directory+"\\PDFs")
                TotalFiles+=1
        print("Total Files : ",TotalFiles)
    except Exception as e:
        print(e)
        print("Error: Aborting")
    finally:
        word.Quit()
    
    #Extracting data of suspicious Institutions
    Edu_data = pd.read_excel('Suspicious Education Institutions -Universities List.xls', skiprows=2) 

    #Extracting data of suspicious employee
    Emp_data = pd.read_excel('Suspicious List of Emp_19-07-2021.xlsx',skiprows=0)

    sorted_data=dict()
    sorted_data['Suspicious Institute']=frd.getList(Edu_data['Name of the Institute'])
    sorted_data['Suspicious Employer']=frd.getList(Emp_data['Suspicious Employer Name'])

    sorted_data=dict()
    sorted_data['Suspicious Institute']=frd.clean(Edu_data['Name of the Institute'])
    sorted_data['Suspicious Employer']=frd.clean(Emp_data['Suspicious Employer Name'])
    acronym=[]
    for name in sorted_data['Suspicious Institute']:
        filtered_sentence = remove_stopwords(name)
        s=""
        n=""
        for x in filtered_sentence.split():
            if(x[0].isalpha()==False or x[0]=='(' ):
                break
            s+=x[0].upper()
            n+=x+" "
            if(',' in x or '/' in x):
                break

        if(len(s)>3) and s not in acronym:  
            acronym.append(s.lower())
    # inst_abbr=[i for i in set(acronym+InstituteAbbr)]
    inst_abbr = list(set(acronym))

    #To get acronyms from Employer list
    acronym=[]
    for name in sorted_data['Suspicious Employer']:
        filtered_sentence = remove_stopwords(name)
        s=""
        n=""
        for x in filtered_sentence.split():
            if(x[0].isalpha()==False or x[0]=='('):
                break

            s+=x[0].upper()
            n+=x+" "
            if(',' in x or '/' in x):
                break
        if(len(s)>3) and (s not in acronym):  
            acronym.append(s.lower())
    # emp_abbr=[i for i in set(acronym+EmployerAbbr)]
    emp_abbr = list(set(acronym))


    # In[ ]:

    vv=30
    flag=0
    ll=Label(gui,text="Checking..",font=("Times New Roman Bold",15),fg='Red',bg='aqua')
    ll.place(x=0,y=vv)
    for file in os.listdir(directory+'\\PDFs'):
        vv+=30
        l1=Label(gui,text=str(file),font=("Times New Roman Bold",15),fg='Red',bg='aqua')
        l1.place(x=0,y=vv)
        if file.endswith(".pdf")=='False' :
            continue
        content = frd.pdfExtract(directory+'\\PDFs\\'+file)
        category_org = frd.ORG(content)
        matchDict={'Direct':[],'Acronym':[],'Fuzzy':[]}
        category_org=list(set(category_org)) 
        #Direct match
        for i in category_org:
            if i in sorted_data["Suspicious Institute"]:
                matchDict['Direct'].append("Institute DIRECT MATCH: "+i)
                flag=1
            if i in sorted_data["Suspicious Employer"]:
                matchDict['Direct'].append("Employer DIRECT MATCH: "+i)

        #Acronyms match
        for name in category_org:
            if name in inst_abbr:
                matchDict['Acronym'].append(name)
        for name in category_org:
            if name in emp_abbr:
                matchDict['Acronym'].append(name)


         #Fuzzy match       
        for pred_org in category_org:
            for sus_inst in sorted_data["Suspicious Institute"]:
                ratio=fuzz.ratio(pred_org,sus_inst)
                if(ratio>95):
                    matchDict['Fuzzy'].append("Institute [FUZZY MATCH]"+pred_org+"->"+sus_inst+" : "+str(ratio))

            for sus_emp in sorted_data["Suspicious Employer"]:
                ratio=fuzz.token_sort_ratio(pred_org,sus_emp)
                if(ratio>95):

                    matchDict['Fuzzy'].append("Employer [FUZZY MATCH]- "+pred_org+"->"+sus_emp+" : "+str(ratio))
                    
                    
        document = Document()
        document.add_heading('Result', 0)
        for key in matchDict.keys():
            if(len(matchDict[key])>0):
                if(flag==0):
                    document.add_paragraph("<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>")
                    document.save('Result.docx')
                    document.add_paragraph(str(file))
                    document.save('Result.docx')
                flag=1
                print(key,' : ',matchDict[key])
                res=str(key)+':'+str(matchDict[key])
                document.add_paragraph(str(res))
                document.save('Result.docx')
                try:
                    if not os.path.exists(directory+'\\FraudPdfs\\'+file):
                        shutil.move(directory+'\\PDFs\\'+file, directory+'\\FraudPdfs\\')
                except Exception as e:
                    print(e)
    document.save('Result.docx')
    l2=Label(gui,text="The results are saved in doc.docx in the same folder ",font=("Times New Roman Bold",15),fg='Red',bg='aqua')
    l2.place(x=0,y=vv+30)
    
def selectFolder():
    directory = filedialog.askdirectory()
    run(directory)
B = tkinter.Button(gui, text ="Browse",font=("Times New Roman Bold", 10),height=1,width=15, command = selectFolder)
B.place(x=160,y=2)

gui.mainloop()


AarthiElangovan[6_0].pdf
AarthiElangovan[6_0].pdf
Ajay Kumar Tableau 3 Year Exp.pdf
AmitTiwari_695528_-03_07-_1.pdf
AnkurSharma_692994_-05_06-_1.pdf
AvinashMahalle-RNSNgp.pdf
BhagyashriPathade_690554_-03_05-_1.pdf
dnyaneshwar.pdf
DushyantSupare_694390_-02_08-_1.pdf
ESHWERRAO[3_0].pdf
GaneshPremrajPatil[6_0].pdf
GauravDipakKhandar_697020_-03_03-_1.pdf
Total Files :  12
Direct  :  ['Employer DIRECT MATCH: rig technologies private limited']
Fuzzy  :  ['Employer [FUZZY MATCH]- rig technologies private limited->rig technologies private limited : 100']
Acronym  :  ['idea']
Direct  :  ['Employer DIRECT MATCH: dns']
Fuzzy  :  ['Employer [FUZZY MATCH]- dns->dns : 100']
